In [6]:
import os
import glob
import scipy
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
def load_mat(file_path):
    filename = os.path.basename(file_path).split(".")[0] # eg.B0005
    data = scipy.io.loadmat(file_path)
    # print(type(data)) # dict
    # print(data.keys()) # dict_keys(['__header__', '__version__', '__globals__', 'B0005'])
    # print(data["B0005"][0][0][0][0].shape) # (616,)，实验持续了616个循环
    # print(type(data["B0005"][0][0][0][0][0])) # 用scipy读取mat文件，读取到结构体返回 numpy.void 类型数据

    cols = data[filename][0][0][0][0]
    
    data = []
    
    for col in cols:
        _ = {}
        parameter = {}
        keys = list(col[3][0][0].dtype.fields.keys())
        if str(col[0][0]) != "impedance":
            for idx, key in enumerate(keys):
                parameter[key] = list(col[3][0][0][idx][0])

        operation_type = str(col[0][0])
        temperature = str(col[1][0][0])
        time = str(datetime(
            int(col[2][0][0]), int(col[2][0][1]), int(col[2][0][2]), int(col[2][0][3]), int(col[2][0][4]), int(col[2][0][5])
        ))
        _["type"], _["temperature"], _["time"], _["data"] = operation_type, temperature, time, parameter
        data.append(_)

    return data

In [8]:
def extract_charge(data, n):
    """提取充电过程数据"""
    vit = []
    bcycle = len(data)
    # 最后一组数据的测量点过少，舍弃
    for i in range(0, bcycle - 1):
        if data[i]["type"] == "charge":
            end = len(data[i]["data"]["Voltage_measured"])
            le = end % n

            vTemp = np.array(data[i]["data"]["Voltage_measured"][0:end-le])
            vTemp = vTemp.reshape(int(len(vTemp)/n), -1, order="F")
            vTemp = vTemp.mean(axis=0)

            iTemp = np.array(data[i]["data"]["Current_measured"][0:end-le])
            iTemp = iTemp.reshape(int(len(iTemp)/n), -1, order="F")
            iTemp = iTemp.mean(axis=0)

            tTemp = np.array(data[i]["data"]["Temperature_measured"][0:end-le])
            tTemp = tTemp.reshape(int(len(tTemp)/n), -1, order="F")
            tTemp = tTemp.mean(axis=0)

            vit.append(np.hstack((vTemp, iTemp, tTemp)))

    vit = np.array(vit)
    return vit

def extract_discharge(data):
    """"提取放电过程数据"""
    bcycle = len(data)
    capacity = []
    for i in range(0, bcycle):
        if data[i]["type"] == "discharge":
            capacity.append(data[i]["data"]["Capacity"][0])
    return capacity

def sequence_target_generator(vit, capacity, capacity_init):
    """
    returns
    ----------
    X, y
    """
    samples = dict()
    comp = len(vit) - len(capacity)
    assert comp > 0, f"len(vit) = {len(vit)}, len(capacity) = {len(capacity)}"
    samples["sequences"] = vit
    samples["targets"] = np.array([capacity_init for i in range(comp)] + capacity)
    return samples

In [9]:
dataset_names = ["B0005", "B0006", "B0007"]
dataset_capacity_init = [1.86, 2.04, 1.89]
dataset_path = "../data/nasa/"
dataset_files = []
for name in dataset_names:
    dataset_files.append(os.path.join(dataset_path, name + ".mat"))
print(dataset_files)

b5 = load_mat(dataset_files[0])
b5_charge, b5_discharge = extract_charge(b5, 10), extract_discharge(b5)
b5_dataset = sequence_target_generator(b5_charge, b5_discharge, dataset_capacity_init[0])

b6 = load_mat(dataset_files[1])
b6_charge, b6_discharge = extract_charge(b6, 10), extract_discharge(b6)
b6_dataset = sequence_target_generator(b6_charge, b6_discharge, dataset_capacity_init[1])

b7 = load_mat(dataset_files[2])
b7_charge, b7_discharge = extract_charge(b7, 10), extract_discharge(b7)
b7_dataset = sequence_target_generator(b7_charge, b7_discharge, dataset_capacity_init[2])

# print(type(b5), len(b5)) # list, 616
# print(type(b6), len(b6)) # list, 616
# print(type(b7), len(b7)) # list, 616

# plt.plot(b5_discharge)
# plt.plot(b6_discharge)
# plt.plot(b7_discharge)
# plt.legend(["B0005", "B0006", "B0007"])

path = "../data/nasa/"
for idx, data in enumerate((b5_dataset, b6_dataset, b7_dataset)):
    save_path = os.path.join(path, dataset_names[idx].lower() + ".npy")
    print(f"save data to path: {save_path}")
    np.save(save_path, data)

['../data/nasa/B0005.mat', '../data/nasa/B0006.mat', '../data/nasa/B0007.mat']
save data to path: ../data/nasa/b0005.npy
save data to path: ../data/nasa/b0006.npy
save data to path: ../data/nasa/b0007.npy
